In [1]:
!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 88.5 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.8/558.8 kB 22.9 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.33.1
    Uninstalling huggingface-hub-0.33.1:
      Successfully uninstalled huggingface-hub-0.33.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.52.4
    Uninstalling transformers-4.52.4:
      Successfully uninstalled transformers-4.52.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.5.1 which is incompatible.


In [9]:
!pip install dotenv

In [3]:
def read_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    lines = [line.strip() for line in lines] 
    return lines
#read_file("/content/1EVChargingStation.txt")

## Load "Llama-3.2-3B-Instruct" from Hugging Face

In [4]:
from huggingface_hub import login
login(new_session=False)

In [5]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
# Check for CUDA availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-3B-Instruct").to(device)

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

2025-07-31 22:54:25.097428: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754002465.314331      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754002465.369544      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [7]:
model.to(device)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((3072,), eps=1e-05)
    (rotary_emb

### Generate Questions Using (Llama-3.2-3B-Instruct) : 

In [10]:
from rich import print
import os
from openai import OpenAI
from datasets import Dataset, DatasetDict, load_dataset
import json
from dotenv import load_dotenv
load_dotenv()
#Open_router_API_KEY = os.getenv('OPENROUTER_API_KEY')

topic = "electric vehicle charging stations"
n_questions = 2

# 2. Questions Generation

QUESTION_PROMPT_TEMPLATE = """\
Given a topic, generate {n_questions} questions that could be asked about that topic. Your response should be in a list format.
The topic is: {topic} 
The list must be without numbers. The questions should be separated by a newline character. There must be no other text than the list.
"""
def generate_questions( topic, n_questions):
  
    prompt = QUESTION_PROMPT_TEMPLATE.format(topic=topic, n_questions=n_questions)
    messages = [ {"role": "user", "content": prompt}, ]
    inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",).to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=40)
    print(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))
    return tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:])

def question_generator( preprocessed_topics, n_question):
    tasks = [generate_questions( subtopic, n_question) for subtopic in preprocessed_topics]
    question_list = tasks
    return question_list
preprocessed_topics=read_file("/kaggle/input/ev-test/1EVChargingStation.txt")
question_list = question_generator( preprocessed_topics, n_questions)
print(question_list)

question_list_formatted = []
for question_set in question_list:
    question_list_formatted.extend([question.strip() for question in question_set.split("\n") if question])
len(question_list_formatted)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


What are the key elements that should be included in a project report?
How does a well-structured project report contribute to the success of a project?<|eot_id|>

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


What is the primary purpose of a document, and how does it differ from other forms of written communication?

How does the structure and content of a document contribute to its overall purpose and effectiveness in conveying 
information

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

What is the duration of a moratorium period for a loan with a value of 6?

What is the length of a repayment period for a loan with a value of 54?<|eot_id|>

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


What are the limitations of the project report's advice, and how should they be considered in real-world project 
implementation?

How do the views expressed in the project report align with the actual costs and requirements of

[
    'What are the key elements that should be included in a project report?\nHow does a well-structured project 
report contribute to the success of a project?<|eot_id|>',
    'What is the primary purpose of a document, and how does it differ from other forms of written 
communication?\n\nHow does the structure and content of a document contribute to its overall purpose and 
effectiveness in conveying information',
    'What are the primary factors that influence the installation cost of an electric vehicle charging 
station?\n\nHow do the charging speed and power requirements of different types of electric vehicles impact the 
design and installation of charging stations?',
    'What are the key aspects that a pre-feasibility report typically covers for a project?\n\nHow does a 
pre-feasibility report help potential entrepreneurs in identifying viable projects for investment?<|eot_id|>',
    'How can you customize the project capacity and project cost to fit the specific needs of your 
organization?\n\nWhat types of projects can be prepared as a report, and are there any specific requirements or 
formats that you',
    'What is the address of the Sidhivinayak Building in Lucknow?\nIs Sidhivinayak Building located at 27/1/B, 
Gokhlley Marg, Luck',
    'What is the primary objective of the Multi Disciplinary Training Centre at Gandhi Darshan Rajghat, New 
Delhi?\n\nHow does the training centre at Gandhi Darshan Rajghat contribute to the overall development',
    'How do I create a new email account?\nWhat are the best practices for writing effective email subject 
lines?<|eot_id|>',
    'What is the purpose of the websiteudyami.org.in?\nWhat services or resources are offered by the 
websiteudyami.org.in?<|eot_id|>',
    'What is the purpose of the given phone numbers?\nHow can one contact the person or organization associated 
with the given phone numbers?<|eot_id|>',
    'What are the primary components that enable an electric vehicle to move, and how do they interact with each 
other?\n\nHow do electric vehicles with solar panels or electric generators differ from those that rely solely on 
a',
    'What are the different types of connectors used for charging electric vehicles at EV charging stations?\n\nHow
do EV charging stations typically measure and track the amount of energy being transferred to an electric vehicle 
during the charging process',
    'What are the key factors that contribute to the increasing demand for electric vehicle (EV) charging stations 
in countries with high oil prices and limited energy resources?\n\nHow do EV charging stations benefit the 
environment, and',
    'Will the increased number of charging stations be sufficient to cater to the growing demand of electric 
vehicles, and how will this impact the overall charging infrastructure?\n\nHow will governments and private 
companies collaborate to establish a comprehensive',
    'Is recycling always the most environmentally friendly option?\n\nIs it possible for some environmental 
practices to have unintended negative consequences on the environment?<|eot_id|>',
    'How will the new project create jobs in both urban and rural areas of the country?\nWhat measures will be 
taken to ensure that the benefits of employment are shared equitably among all segments of the population?',
    'What are the key factors that a young entrepreneur should consider when deciding whether to install an EV 
charging station in their locality?\n\nHow can a young entrepreneur ensure that their EV charging station is 
profitable and viable in',
    "What is the primary objective of the power ministry's EV policy initiative regarding the rollout plan?\n\nHow 
does the power ministry plan to expand its EV initiative beyond the first phase, targeting specific regions and 
cities?",
    "What factors are expected to drive the growth of the electric vehicle charging infrastructure market in India 
during the forecast period?\n\nHow will the government's policies and re

241

In [16]:
print(len(question_list))

115

In [17]:
# 2. Save the formatted list to a file
output_file_path = "/kaggle/working/generated_questions.txt" # Standard Kaggle output directory

try:
    with open(output_file_path, 'w', encoding='utf-8') as f:
        for question in question_list_formatted:
            f.write(question + '\n')
    print(f"Successfully saved {len(question_list_formatted)} questions to {output_file_path}")

except Exception as e:
    print(f"An error occurred while saving the file: {e}")


Successfully saved 241 questions to /kaggle/working/generated_questions.txt

In [18]:
question_list_formatted = []
for question_set in question_list:
    question_list_formatted.extend([question.strip() for question in question_set.split("\n") if question])
len(question_list_formatted)

241

In [20]:
# 3. Responses Generation
RESPONSE_PROMPT_TEMPLATE = """\
Given a question, generate 2 responses that could be given to that question. Your response should be in a list format.

The question is: {question}

The list must be in the format:

RESPONSE A: Response A text here
RESPONSE B: Response B text here
"""
def generate_responses(question):
    prompt = RESPONSE_PROMPT_TEMPLATE.format(question=question)
    messages = [ {"role": "user", "content": prompt}, ]
    inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",).to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=40)
    print(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))
    return tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:])
    
def response_generator(question_list):
    tasks = [generate_responses(question) for question in question_list]
    response_list = tasks
    return response_list

question_response_list = response_generator(question_list_formatted)
question_response_pair_list = []
for question, response_set in zip(question_list_formatted, question_response_list):
    question_response_pair_list.append(
        {
            "question": question,
            "responses": {
                "response_a": {"response": response_set.split("RESPONSE B:")[0].replace("RESPONSE A:", "").strip()},
                "response_b": {"response": response_set.split("RESPONSE B:")[-1].split("\n\n")[0].strip()}
            },
        }
    )

with open('synthetic_data.jsonl', 'w') as f:
    for item in question_response_pair_list:
        f.write(json.dumps(item))
        f.write('\n')


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


RESPONSE A: 
A comprehensive project report should include the following key elements:
1. Executive Summary: A brief overview of the project, its objectives, and the main findings.
2. Introduction:

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


RESPONSE A: A well-structured project report contributes to the success of a project by providing stakeholders with
a clear understanding of the project's objectives, scope, timeline, and outcomes. It serves as

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

I need more information to provide accurate responses. The length of a repayment period for a loan depends on 
various factors such as the interest rate, loan type, and repayment terms. However, I can provide

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


RESPONSE A: 
The limitations of a project report's advice can be attributed to the constraints of the project scope, timelines, 
and resources. In real-world project implementation, these limitations should be considered

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


RESPONSE A: The views expressed in the project report should be regularly reviewed and updated to ensure they 
remain aligned with the actual costs and requirements of the project. This can be achieved through ongoing 
monitoring and

In [ ]:
messages = [
    {
        "role": "user",
        "content": "Hello!"
    },
    {
        "role": "assistant",
        "content": "Hello! How can I help you today?"
    },
]

response = client.chat.completions.create(
    model="nvidia/nemotron-4-340b-reward",
    messages=messages,)
print(response)
print(response.choices[0].logprobs.content)

def get_scores_from_response(openai_response_template):
    logprobs = openai_response_template.choices[0].logprobs.content
    score_dict = {}
    for score in logprobs:
        score_dict[score.token] = score.logprob
    return score_dict

print(get_scores_from_response(response))

def get_response_and_scores(client, model, question, response_content):
    messages = [
        {
            "role": "user",
            "content": question
        },
        {
            "role": "assistant",
            "content": response_content
        },
    ]

    response = client.chat.completions.create(
        model=model,
        messages=messages,
    )

    scores = get_scores_from_response(response)
    return scores

question_response_score_list = question_response_pair_list.copy()

def process_question_response_pairs(client, model, question_response_score_list):
    tasks = []
    for question_response_pair in question_response_score_list:
        question = question_response_pair["question"]

        task_a = get_response_and_scores(client, model, question, question_response_pair["responses"]["response_a"]["response"])
        task_b = get_response_and_scores(client, model, question, question_response_pair["responses"]["response_b"]["response"])

        tasks.append((task_a, question_response_pair, "response_a"))
        tasks.append((task_b, question_response_pair, "response_b"))

    results = [task[0] for task in tasks]

    for i, (result, task_info) in enumerate(zip(results, tasks)):
        _, question_response_pair, response_key = task_info
        question_response_pair["responses"][response_key].update(result)

process_question_response_pairs(client, "nvidia/nemotron-4-340b-reward", question_response_score_list)

threshold = 3.0

with open(f'synthetic_data_with_scores_filtered-{threshold}.jsonl', 'w') as f:
    for item in question_response_score_list:
        question = item["question"]
        response_a = item["responses"]["response_a"]
        response_b = item["responses"]["response_b"]
        response_a["question"] = question
        response_b["question"] = question
        if response_a["helpfulness"] < threshold and response_b["helpfulness"] < threshold:
            continue
        f.write(json.dumps(response_a))
        f.write('\n')
        f.write(json.dumps(response_b))
        f.write('\n')


with open(f'synthetic_data_with_scores_filtered-{threshold}.jsonl', 'r') as f:
    data = [json.loads(line) for line in f]
dataset = Dataset.from_list(data)
dataset_dict = DatasetDict({"train": dataset})
dataset_dict.push_to_hub("mervinpraison/preference-dataset-prep")